# [모의 캐글-의료] 흉부 CT 코로나 감염 여부 분류

- 데이터 증식시켜보기

## Import Libraries

In [ ]:
# !apt-get update && apt-get install -y python3-opencv

In [ ]:
# !pip install opencv-python

In [ ]:
# !pip install pandas

In [ ]:
# !pip install sklearn

In [1]:
import os, torch, copy, cv2, sys, random
# from datetime import datetime, timezone, timedelta
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

# Set Arguments & hyperparameters

In [2]:
# 시드(seed) 설정

RANDOM_SEED = 2022

torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

In [3]:
# parameters

### 데이터 디렉토리 설정 ###
DATA_DIR= 'data'
NUM_CLS = 2  # binary clf 라서 2

EPOCHS = 50 # 30
BATCH_SIZE = 32
LEARNING_RATE = 0.0005
EARLY_STOPPING_PATIENCE = 20 # 10
INPUT_SHAPE = 256  # 128  # img shape을 resize해서 넣을건데 그걸 어떻게 할 것인지


os.environ["CUDA_VISIBLE_DEVICES"]="0"
# DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE = torch.device('cuda')

# Dataloader
: 원본 이미지 shape (384, 384, 3)

## Data Augmentation 
 - 라이브러리 자체 Augmentation 모듈도 있지만 Albumentation이 연산 속도가 빠르대. 
 - 빨라서 캐글러들이 많이 선택한대서 나도 이걸로 선택해봄

In [ ]:
# !pip install albumentations

In [4]:
import albumentations as A
from albumentations.pytorch import transforms as albu_transforms

In [5]:
class CustomDataset(Dataset):  # albu 넣어보기
    def __init__(self, data_dir, mode, input_shape, transform=None):
        self.data_dir = data_dir
        self.mode = mode
        self.input_shape = input_shape
        
        # Loading dataset
        self.db = self.data_loader()
        
        # Dataset split
        if self.mode in ['train', 'aug']:
            self.db = self.db[:int(len(self.db) * 0.9)]  # 앞 90% 불러오기
        elif self.mode == 'val':
            self.db = self.db[int(len(self.db) * 0.9):]  # 뒤 10% 불러오기
            self.db.reset_index(inplace=True)
        else:
            print(f'!!! Invalid split {self.mode}... !!!')
        
        self.transform = transform
        if self.transform:
            self.transform = transform
        else:
            # self.transform = A.Compose([A.Resize(256,256),
            #                             A.Normalize(mean=[0.485, 0.456, 0.406],
            #                                         std=[0.229, 0.224, 0.225],),
            #                             albu_transforms.ToTensorV2(),
            #                            ])
            self.transform = transforms.Compose([transforms.Resize(256),  # 위에서 설정한 값으로 이미지 리사이즈
                                             transforms.ToTensor(),  # 이미지 텐서화
                                             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
            
    def data_loader(self):  # csv 파일 불러오기
        print('Loading ' + self.mode + ' dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        
        # (COVID : 1, No : 0)
        db = pd.read_csv(os.path.join(self.data_dir, 'train.csv'))
        return db

    def __len__(self):
        return len(self.db)

    def __getitem__(self, index):  # 각 데이터의 이미지 텐서, 라벨값 반환
        data = copy.deepcopy(self.db.loc[index])  # index번째 이미지 불러오기 -> 복사본 만들기(찐 데이터에 영향ㄴㄴ)
        
        label = data['COVID']
        file_path = os.path.join(self.data_dir,'train',data['file_name'])
        # Loading image
        cvimg = cv2.imread(file_path)
        cvimg = cv2.cvtColor(cvimg, cv2.COLOR_BGR2RGB)
        
        if not isinstance(cvimg, np.ndarray):
            raise IOError("Fail to read %s" % data['file_name'])  # 혹시 안되면 에러 띄우기

        # Preprocessing images
        if self.mode == 'aug':  # self.transform:
            cvimg = self.transform(image=cvimg)['image']
            # augmented = self.transform(image=cvimg)
            # cvimg = augmented['image']
        else:
            # cvimg = self.transform(image=cvimg)['image']
            cvimg = self.transform(Image.fromarray(cvimg))

        return cvimg, label  # 텐서로 변환된 이미지와 label 반환

### Albumentation

In [6]:
albumentations_transform = A.Compose([
    A.Resize(300, 300), # 256, 256
    A.RandomCrop(256, 256), # 224, 224
    A.HorizontalFlip(p=0.5),  # p의 확률로 좌우 반전
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
    A.RandomBrightnessContrast(p=0.3),
    A.RandomRotate90(p=0.2),
    # A.Blur(),
    # A.VerticalFlip(),
    
    albu_transforms.ToTensorV2()
])

# Model

## CNN

In [7]:
import torch.nn.functional as F

class custom_CNN(nn.Module):
    def __init__(self, num_classes):  # 레이어 선언
        super(custom_CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=25, kernel_size=5)
        
        self.fc1 = nn.Linear(in_features=25*61*61, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=num_classes)
        
        self.softmax = nn.Softmax(dim=1)
        self.dropout = nn.Dropout(p=0.5)
        
    def forward(self, x):  # 레이어 쌓기
        x = self.pool(F.relu(self.conv1(x))) # (32, 3, 128, 128) -> (32, 8, 62, 62)
        # x = self.dropout(x)
        x = self.pool(F.relu(self.conv2(x))) # (32, 8, 62, 62) -> (32, 25, 29, 29)
        # x = self.dropout(x)
        
        # print(x.shape)
        
        x = torch.flatten(x,1)
        # print(x.shape)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        output = self.softmax(x)
        
        return output
    

#### 이런 식으로 class 테스트 가능
custom_CNN forward() 안의 print문 주석 해제 후 실행해보기

In [ ]:
model = custom_CNN(2)
x = torch.randn(3, 3, 256, 256)
model(x)

In [ ]:
25*61*61

# Utils
: 학습에 필요한 클래스들
## EarlyStopper
- 일정 횟수patience 이상 CV loss 개선이 없을 경우 학습을 중단

In [8]:
class LossEarlyStopper():
    """Early stopper
    
    Attributes:
        patience (int): loss가 줄어들지 않아도 학습할 epoch 수
        patience_counter (int): loss 가 줄어들지 않을 때 마다 1씩 증가, 감소 시 0으로 리셋
        min_loss (float): 최소 loss
        stop (bool): True 일 때 학습 중단

    """

    def __init__(self, patience: int)-> None:
        self.patience = patience

        self.patience_counter = 0
        self.min_loss = np.Inf
        self.stop = False
        self.save_model = False

    def check_early_stopping(self, loss: float)-> None:
        """Early stopping 여부 판단"""  
        # if self.save_model:
        #     print("#"*20, "계속 저장중인지 확인용", "#"*20)
        self.save_model = False
        
        if self.min_loss == np.Inf:
            self.min_loss = loss
            return None

        elif loss > self.min_loss:
            self.patience_counter += 1
            msg = f"Early stopping counter {self.patience_counter}/{self.patience}"

            if self.patience_counter == self.patience:
                self.stop = True
                
        elif loss <= self.min_loss:
            self.patience_counter = 0
            self.save_model = True
            msg = f"Validation loss decreased {self.min_loss:4f} -> {loss:4f}  (-{self.min_loss - loss :4f})"
            self.min_loss = loss
        
        print('='*15, '>\t', msg)
        # print(msg)

## Trainer
- Epoch 별 학습 및 검증 절차
- train_epoch / validate_epoch (미니배치 단위 절차)

In [9]:
class Trainer():
    """ epoch에 대한 학습 및 검증 절차 정의"""
    
    def __init__(self, loss_fn, model, device, metric_fn, optimizer=None, scheduler=None):
        """ 초기화
        """
        self.loss_fn = loss_fn
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.metric_fn = metric_fn

    def train_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 학습 절차"""
        
        self.model.train()
        train_total_loss = 0
        target_lst = []
        pred_lst = []
        prob_lst = []

        for batch_index, (img, label) in enumerate(dataloader):
            img = img.to(self.device)
            label = label.to(self.device).float()
            
            pred = self.model(img)
            
            loss = self.loss_fn(pred[:,1], label)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            self.scheduler.step()
            
            train_total_loss += loss.item()
            prob_lst.extend(pred[:, 1].cpu().tolist())
            target_lst.extend(label.cpu().tolist())
            pred_lst.extend(pred.argmax(dim=1).cpu().tolist())
        self.train_mean_loss = train_total_loss / batch_index
        self.train_score, f1 = self.metric_fn(y_pred=pred_lst, y_answer=target_lst)
        msg = f'Epoch {epoch_index}, Train loss: {self.train_mean_loss}, Acc: {self.train_score}, F1-Macro: {f1}'
        print(msg)

    def validate_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 검증 절차
        """
        self.model.eval()
        val_total_loss = 0
        target_lst = []
        pred_lst = []
        prob_lst = []

        for batch_index, (img, label) in enumerate(dataloader):
            img = img.to(self.device)
            label = label.to(self.device).float()
            pred = self.model(img)
            
            loss = self.loss_fn(pred[:,1], label)
            val_total_loss += loss.item()
            prob_lst.extend(pred[:, 1].cpu().tolist())
            target_lst.extend(label.cpu().tolist())
            pred_lst.extend(pred.argmax(dim=1).cpu().tolist())
            print(pred_lst)  # 예측값 보려고 추가
            
        self.val_mean_loss = val_total_loss / batch_index
        self.validation_score, f1 = self.metric_fn(y_pred=pred_lst, y_answer=target_lst)
        msg = f'Epoch {epoch_index}, Val loss: {self.val_mean_loss}, Acc: {self.validation_score}, F1-Macro: {f1}'
        print(msg)
        

## Metrics
메트릭 점수 계산 : 성능 반환

In [10]:
from sklearn.metrics import accuracy_score, f1_score

def get_metric_fn(y_pred, y_answer):
    """ 성능을 반환하는 함수"""
    
    assert len(y_pred) == len(y_answer), 'The size of prediction and answer are not same.'
    
    accuracy = accuracy_score(y_answer, y_pred)
    f1 = f1_score(y_answer, y_pred, average='macro')
    
    return accuracy, f1

# Train 학습을 위한 객체 선언

## Load Dataset & Dataloader

In [11]:
# Load dataset & dataloader
train_origin_dataset = CustomDataset(data_dir=DATA_DIR, mode='train', input_shape=INPUT_SHAPE)
train_aug_dataset = CustomDataset(data_dir=DATA_DIR, mode='aug', input_shape=INPUT_SHAPE, transform=albumentations_transform)
validation_dataset = CustomDataset(data_dir=DATA_DIR, mode='val', input_shape=INPUT_SHAPE)


train_dataset = torch.utils.data.ConcatDataset([train_origin_dataset, train_aug_dataset])
# validation_dataset = torch.utils.data.ConcatDataset([validation_dataset])


train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True)

print('Train set samples:',len(train_dataset),  'Val set samples:', len(validation_dataset))

Loading train dataset..
Loading aug dataset..
Loading val dataset..
Train set samples: 1162 Val set samples: 65


## Load model and other utils

In [12]:
# Load Model
model = custom_CNN(NUM_CLS).to(DEVICE)

# # Save Initial Model
# torch.save(model.state_dict(), 'initial.pt')

# Set optimizer, scheduler, loss function, metric function
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler =  optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e5, max_lr=0.0001, epochs=EPOCHS, steps_per_epoch=len(train_dataloader))
loss_fn = nn.BCELoss()
metric_fn = get_metric_fn


# Set trainer
trainer = Trainer(loss_fn, model, DEVICE, metric_fn, optimizer, scheduler)

# Set earlystopper
early_stopper = LossEarlyStopper(patience=EARLY_STOPPING_PATIENCE)

In [ ]:
model

## epoch 단위 학습 진행
- Trainer의 train_epoch, vaDATA_DIRd_epoch를 에폭마다 실행
- CV los에 따른 early stop 여부 확인
- Loss 개선 시 가중치 저장

In [13]:
for epoch_index in tqdm(range(EPOCHS)):  # tqdm : 진행상황을 가시적으로 보여주는 모듈

    trainer.train_epoch(train_dataloader, epoch_index)
    trainer.validate_epoch(validation_dataloader, epoch_index)

    # early_stopping check
    early_stopper.check_early_stopping(loss=trainer.val_mean_loss)

    if early_stopper.stop:
        print('Early stopped')
        break

    if early_stopper.save_model:
        check_point = {  # 모델 가중치 적용
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'scheduler': scheduler.state_dict()
        }
        torch.save(check_point, 'best.pt')


  0% 0/50 [00:00<?, ?it/s]

Epoch 0, Train loss: 0.704361680481169, Acc: 0.5292598967297762, F1-Macro: 0.5258179571977324
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0]


  2% 1/50 [00:58<48:01, 58.81s/it]

[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0]
Epoch 0, Val loss: 1.0958735346794128, Acc: 0.6615384615384615, F1-Macro: 0.6366869918699187
Epoch 1, Train loss: 0.6636489729086558, Acc: 0.6144578313253012, F1-Macro: 0.6092616611423356
[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1]


  4% 2/50 [01:53<46:04, 57.60s/it]

...

[0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
[0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
Epoch 33, Val loss: 0.5188532192260027, Acc: 0.7384615384615385, F1-Macro: 0.7321212121212122
=============== >	 Early stopping counter 18/20
Epoch 34, Train loss: 0.26874623571832973, Acc: 0.8726333907056799, F1-Macro: 0.8717580840966781
[1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0]


 70% 35/50 [39:38<16:19, 65.32s/it]

[1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0]
[1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0]
Epoch 34, Val loss: 0.6016448140144348, Acc: 0.8, F1-Macro: 0.7998104714522627
=============== >	 Early stopping counter 19/20
Epoch 35, Train loss: 0.27435752376914024, Acc: 0.8855421686746988, F1-Macro: 0.885037361035174
[0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1]


 70% 35/50 [40:44<17:27, 69.84s/it]

[0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1]
[0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0]
Epoch 35, Val loss: 0.4853028245270252, Acc: 0.8153846153846154, F1-Macro: 0.8142857142857143
=============== >	 Early stopping counter 20/20
Early stopped


# Inference
## Load
### 모델 로드

In [14]:
TRAINED_MODEL_PATH = 'best.pt'

### Test dataset Load

In [15]:
class TestDataset(Dataset):
    def __init__(self, data_dir, input_shape):
        self.data_dir = data_dir
        self.input_shape = input_shape
        
        # Loading dataset
        self.db = self.data_loader()
        
        # Transform function
        self.transform = transforms.Compose([transforms.Resize(self.input_shape),
                                             transforms.ToTensor(),
                                             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def data_loader(self):
        print('Loading test dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        
        db = pd.read_csv(os.path.join(self.data_dir, 'sample_submission.csv'))
        return db
    
    def __len__(self):
        return len(self.db)
    
    def __getitem__(self, index):
        data = copy.deepcopy(self.db.loc[index])
        
        # Loading image
        cvimg = cv2.imread(os.path.join(self.data_dir,'test',data['file_name']), cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
        if not isinstance(cvimg, np.ndarray):
            raise IOError("Fail to read %s" % data['file_name'])

        # Preprocessing images
        trans_image = self.transform(Image.fromarray(cvimg))

        return trans_image, data['file_name']

In [16]:
# Load dataset & dataloader
test_dataset = TestDataset(data_dir=DATA_DIR, input_shape=INPUT_SHAPE)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

Loading test dataset..


## 추론 진행
: 미니 배치 단위로 추론

### valid 성능 확인용

In [17]:
model.load_state_dict(torch.load(TRAINED_MODEL_PATH)['model'])

train_actual = []
validation_actual = []

train_pred_lst = []
validation_pred_lst = []

model.eval()

# train
with torch.no_grad():
    for batch_index, (img, label) in tqdm(enumerate(train_dataloader)):
        img = img.to(DEVICE)
        pred = model(img)
        train_actual += (list(label.numpy()))
        train_pred_lst.extend(pred.argmax(dim=1).tolist())

# validation
with torch.no_grad():
    for batch_index, (img, label) in tqdm(enumerate(validation_dataloader)):
        img = img.to(DEVICE)
        pred = model(img)
        validation_actual += (list(label.numpy()))
        validation_pred_lst.extend(pred.argmax(dim=1).tolist())

37it [00:57,  1.56s/it]
3it [00:07,  2.35s/it]


### test 성능 확인

In [18]:
model.load_state_dict(torch.load(TRAINED_MODEL_PATH)['model'])

# Prediction
file_lst = []
pred_lst = []
prob_lst = []
model.eval()
with torch.no_grad():
    for batch_index, (img, file_num) in tqdm(enumerate(test_dataloader)):
        img = img.to(DEVICE)
        pred = model(img)
        # print(pred)
        file_lst.extend(list(file_num))
        pred_lst.extend(pred.argmax(dim=1).tolist())
        prob_lst.extend(pred[:, 1].tolist())

4it [00:08,  2.18s/it]


# 결과 저장

In [20]:
accuracy = accuracy_score(validation_actual, validation_pred_lst)
accuracy

0.7692307692307693

In [21]:
# 시간 정보, F1 score 활용해서 이름 생성
from datetime import datetime, timedelta, timezone 

sub_filename = f'[{accuracy:.4f}]_' + datetime.now(timezone(timedelta(hours=9))).strftime("%y%m%d-%H%M%S") + '.csv'

In [22]:
df = pd.DataFrame({'file_name':file_lst, 'COVID':pred_lst})
# df.sort_values(by=['file_name'], inplace=True)
df.to_csv('submit/'+sub_filename, index=False)

print(sub_filename)

[0.7692]_220215-112510.csv
